In [0]:
from pyspark.sql.functions import col, regexp_replace

# Read from Bronze Delta Table
df_bronze = spark.read.format("delta").load("/FileStore/lending/bronze/delta")


+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+--------------------+------------------+------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+-

In [0]:
# Clean & Transform
df_silver = df_bronze.dropna(subset=["loan_amnt", "term", "int_rate"]) \
    .withColumn("int_rate", regexp_replace("int_rate", "%", "").cast("double")) \
    .withColumn("loan_amnt", col("loan_amnt").cast("double")) \
    .withColumn("annual_inc", col("annual_inc").cast("double")) \
    .withColumn("dti", col("dti").cast("double")) \
    .withColumn("emp_length", regexp_replace("emp_length", "\\+ years| years|< 1 year", "")) \
    .withColumn("emp_length", regexp_replace("emp_length", "n/a", "0")) \
    .withColumn("emp_length", col("emp_length").cast("int"))



In [0]:
# Save as Delta Table in Silver Layer
df_silver.write.format("delta").mode("overwrite").save("/FileStore/lending/silver/delta")

In [0]:
# Preview Transformed Data
df_silver.show(5)

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+--------------------+------------------+------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+-